In [122]:
import numpy as np
import tabulate

In [201]:
BLANK_VAL = -1
STEP_COUNTER = 0

In [203]:
class Board:
    def __init__(self,vals,blankVal,defaults=None):
        if isinstance(vals,list):
            self.board = np.array(vals)
        else :
            self.board = vals
        self.dim = int(len(vals) ** 0.5)
        assert self.dim == len(vals) ** 0.5
        self.boxDim = int(self.dim ** 0.5)
        assert self.boxDim == self.dim ** 0.5,f'{self.boxDim}, {self.dim ** 0.5}'
        self.digits = set(range(1,self.dim+1))
        self.blank = blankVal
        if defaults is None:
            self.defaults = np.array([val != self.blank for val in self.board])
        else :
            self.defaults = defaults

    def __str__(self):
        vals = np.where(self.board==BLANK_VAL,'',self.board).reshape(self.dim,self.dim)
        return tabulate.tabulate(vals,tablefmt='grid')

    def testFunc(self,func):
        vals = []
        for idx in range(len(self.board)):
            vals.append(func(idx))
        return tabulate.tabulate(np.array(vals).reshape(self.dim,-1),tablefmt='grid')

    def row(self,r):
        start = self.dim*r
        return self.board[start:start+self.dim]

    def col(self,c):
        return self.board[c::self.dim]

    def box(self,b):
        rStart = (b // self.boxDim) * self.boxDim
        cStart = (b % self.boxDim) * self.boxDim
        return np.array([self.board[r*self.dim+c] for r in range(rStart,rStart+self.boxDim) for c in range(cStart,cStart+self.boxDim)])

    def validMove(self,idx,val):
        r = idx // self.dim
        c = idx % self.dim
        b = (r // self.boxDim) * self.boxDim + (c // self.boxDim)
        inRow = val in self.row(r)
        inCol = val in self.col(c)
        inBox = val in self.box(b)
        # print(idx,val,inRow,inCol,inBox)
        return not inRow and not inCol and not inBox

    def isSolved(self):
        for i in range(self.dim):
            if set(self.row(i)) != self.digits:
                return False
            if set(self.col(i)) != self.digits:
                return False
            if set(self.box(i)) != self.digits:
                return False
        return True


    # def solve(self,idx):
    #     print(self)
    #     print(idx)
    #     if self.isSolved():
    #         return True
    #     if idx > len(self.board):
    #         return False
    #     if self.defaults[idx]:
    #         return self.solve(idx+1)
    #     attempt = False
    #     for d in self.digits:
    #         if self.validMove(idx,d):
    #             self.board[idx] = d
    #             attempt = attempt or self.solve(idx+1)
    #     return attempt


def solveBoard(idx,board,verbose=False):
    global STEP_COUNTER
    if verbose:
        print(idx)
        print(board)
    STEP_COUNTER += 1
    if board.isSolved():
        print(f'Solved in {STEP_COUNTER} steps')
        print(board)
        return True
    if idx > len(board.board):
        return False
    if board.defaults[idx]:
        return solveBoard(idx+1,board,verbose)
    attempt = False
    for d in board.digits:
        if board.validMove(idx,d):
            b = Board(np.array(board.board),board.blank,board.defaults)
            b.board[idx] = d
            attempt = attempt or solveBoard(idx+1,b,verbose)
    return attempt

In [208]:
vals = [
    1,2,3,4,
    3,4,1,2,
    2,3,4,1,
    4,1,2,3
]
# from ChatGPT, might not be solvable
vals = [
    5, 3, -1, -1, 7, -1, -1, -1, -1,
    6, -1, -1, 1, 9, 5, -1, -1, -1,
    -1, 9, 8, -1, -1, -1, -1, 6, -1,
    8, -1, -1, -1, 6, -1, -1, -1, 3,
    4, -1, -1, 8, -1, 3, -1, -1, 1,
    7, -1, -1, -1, 2, -1, -1, -1, 6,
    -1, 6, -1, -1, -1, -1, 2, 8, -1,
    -1, -1, -1, 4, 1, 9, -1, -1, 5,
    -1, -1, -1, -1, 8, -1, -1, 7, 9
]
# Easy. 153 steps
vals = [
    -1, 2, 6, -1, -1, -1, -1, -1, 1,
    -1, 4, 5, -1, 7, 1, -1, -1, -1,
    8, -1, 1, -1, -1, 6, -1, 7, -1,
    -1, -1, -1, 1, 3, 2, 6, 8, -1,
    5, 3, 8, -1, -1, -1, -1, 1, 9,
    -1, -1, -1, -1, -1, -1, 7, -1, 4,
    -1, -1, -1, -1, 2, 7, -1, 6, 8,
    -1, 8, -1, 9, 1, 5, -1, -1, 2,
    -1, 5, -1, 6, 8, 3, -1, 9, 7
]
# Medium. 429 steps
vals = [
    -1, -1, -1, 9, 2, -1, -1, -1, -1,
    -1, 4, -1, 8, 5, 1, -1, -1, -1,
    2, 5, 6, -1, -1, 3, -1, 9, 1,
    1, -1, -1, -1, 8, 5, 4, -1, 9,
    -1, 9, 8, 7, 3, -1, 1, 6, 2,
    -1, -1, -1, 2, -1, -1, 5, 3, -1,
    -1, -1, 7, -1, 6, -1, 9, -1, -1,
    9, -1, -1, -1, -1, 2, 6, 8, -1,
    -1, 8, -1, -1, 9, -1, -1, 5, 4
]
# Hard. 1421 steps
vals = [
    3, -1, -1, -1, 9, -1, -1, 7, 1,
    7, -1, -1, 5, -1, 1, 3, -1, -1,
    8, -1, -1, 7, -1, -1, -1, 5, -1,
    -1, -1, 8, -1, -1, 9, -1, 6, -1,
    -1, -1, 2, -1, -1, 4, 5, -1, 8,
    4, 7, 9, -1, -1, -1, -1, 3, -1,
    -1, -1, -1, -1, 3, -1, -1, -1, -1,
    -1, 1, -1, 6, -1, 8, -1, -1, 7,
    2, 5, -1, -1, -1, -1, -1, -1, 3
]
# Extreme. 65,246 steps
vals = [
    3, -1, -1, -1, -1, 9, 8, -1, 1,
    -1, -1, 6, -1, -1, -1, -1, -1, 9,
    8, -1, -1, -1, 7, -1, -1, -1, -1,
    -1, -1, -1, 4, -1, -1, -1, -1, -1,
    5, -1, -1, -1, -1, -1, -1, 6, -1,
    -1, 7, -1, -1, 8, -1, 1, 3, -1,
    -1, 1, -1, -1, -1, -1, 4, -1, -1,
    -1, -1, -1, 2, -1, 3, 9, -1, -1, 
    -1, 5, 4, -1, -1, 1, -1, -1, -1
]

b = Board(vals,BLANK_VAL)
# print(b)
# print(b.row(0))
# print(b.row(1))
# print(b.col(0))
# print(b.col(1))
# print(b.box(0))
# print(b.box(1))
# print(b.box(2))
# print(b.box(3))
# print(b.defaults)
# print(b.digits)
# print(b.isSolved())

# def boxFromIdx(idx):
#     r = idx // b.dim
#     c = idx % b.dim
#     return (r // b.boxDim)*b.boxDim + (c // b.boxDim)

# print(b.testFunc(boxFromIdx))
# print(b.solve(0))
# print(b)

STEP_COUNTER = 0
print(solveBoard(0,b,False))

Solved in 65246 steps
+---+---+---+---+---+---+---+---+---+
| 3 | 4 | 5 | 6 | 2 | 9 | 8 | 7 | 1 |
+---+---+---+---+---+---+---+---+---+
| 7 | 2 | 6 | 1 | 5 | 8 | 3 | 4 | 9 |
+---+---+---+---+---+---+---+---+---+
| 8 | 9 | 1 | 3 | 7 | 4 | 2 | 5 | 6 |
+---+---+---+---+---+---+---+---+---+
| 1 | 6 | 2 | 4 | 3 | 7 | 5 | 9 | 8 |
+---+---+---+---+---+---+---+---+---+
| 5 | 3 | 8 | 9 | 1 | 2 | 7 | 6 | 4 |
+---+---+---+---+---+---+---+---+---+
| 4 | 7 | 9 | 5 | 8 | 6 | 1 | 3 | 2 |
+---+---+---+---+---+---+---+---+---+
| 9 | 1 | 3 | 8 | 6 | 5 | 4 | 2 | 7 |
+---+---+---+---+---+---+---+---+---+
| 6 | 8 | 7 | 2 | 4 | 3 | 9 | 1 | 5 |
+---+---+---+---+---+---+---+---+---+
| 2 | 5 | 4 | 7 | 9 | 1 | 6 | 8 | 3 |
+---+---+---+---+---+---+---+---+---+
True


In [153]:
np.array(vals).reshape(9,9)

array([[ 5,  3, -1, -1,  7, -1, -1, -1, -1],
       [ 6, -1, -1,  1,  9,  5, -1, -1, -1],
       [-1,  9,  8, -1, -1, -1, -1,  6, -1],
       [ 8, -1, -1, -1,  6, -1, -1, -1,  3],
       [ 4, -1, -1,  8, -1,  3, -1, -1,  1],
       [ 7, -1, -1, -1,  2, -1, -1, -1,  6],
       [-1,  6, -1, -1, -1, -1,  2,  8, -1],
       [-1, -1, -1,  4,  1,  9, -1, -1,  5],
       [-1, -1, -1, -1,  8, -1, -1,  7,  9]])

In [104]:
print(b)
idx = 8
val = 2
b.validMove(idx,2)
r = idx // b.dim
c = idx % b.dim
box = (r // b.boxDim) * b.boxDim + (c // b.boxDim)
print(r,b.row(r))
print(c,b.col(c))
print(box,b.box(box))
inRow = val in b.row(r)
inCol = val in b.col(c)
inBox = val in b.box(box)
print(inRow,inCol,inBox)

+---+---+---+---+
| 1 | 2 | 3 | 4 |
+---+---+---+---+
| 3 | 4 | 1 | 2 |
+---+---+---+---+
|   | 3 | 4 | 1 |
+---+---+---+---+
| 4 | 1 | 2 |   |
+---+---+---+---+
8 2 False False True
2 [-1  3  4  1]
0 [ 1  3 -1  4]
2 [-1  3  4  1]
False False False



    """def solve(self,idx,val):
        print(self)
        print(idx,val)
        if idx > len(self.board):
            return False
        if self.defaults[idx]:
            return self.solve(idx+1,val)
        if self.validMove(idx,val):
            self.board[idx] = val
            return self.solve(idx+1,1)
        else :
            print('invalid!')
        if self.isSolved():
            return True
        elif val+1 not in self.digits:
            return False
        else :
            return self.solve(idx,val+1)
        # attempts = False
        # for d in self.digits:
        #     if self.validMove(idx+1,d):
        #         attempts = attempts or self.solve(idx+1,d)
        # return attempts

In [133]:
x = [1,2,3]
y = x[:]
print(id(x))
print(id(y))
print('x',x)
print('y',y)
y[0]=4
print(id(x))
print(id(y))
print('x',x)
print('y',y)

2411156922048
2411156968512
x [1, 2, 3]
y [1, 2, 3]
2411156922048
2411156968512
x [1, 2, 3]
y [4, 2, 3]


In [166]:
x = np.array([1,2,3])
y = np.array(x)
print(id(x), id(y))
print(x,y)
y[0] = 4
print(x,y)
x[0] = 5
print(x,y)

2411159642320 2411159642416
[1 2 3] [1 2 3]
[1 2 3] [4 2 3]
[5 2 3] [4 2 3]


In [143]:
class A:
    def __init__(self,myArr):
        self.arr = np.array(myArr)

    def __str__(self):
        return str(self.arr)

a1 = A([1,2,3])
print(a1) # [1, 2, 3]
print(id(a1),id(a1.arr)) # two different numbers
a1.arr[0] = 4
print(a1,id(a1),id(a1.arr)) # [4,2,3] then the same #s as above

a2 = A(a1.arr)
print(a2,id(a2),id(a2.arr),id(a1),id(a1.arr)) # [4,2,3], random #, id(a1.arr)
a2.arr[0] = 8
print(a2,id(a2),id(a2.arr),a1,id(a1),id(a1.arr)) # [8,2,3], random #, id(a1.arr), [8,2,3] same #s as above

a1.arr[0] = 6
print(a2,id(a2),id(a2.arr),a1,id(a1),id(a1.arr)) # [6,2,3], random #, id(a1), [6,2,3], same #s as above

a3 = A(a1.arr[:])
print(a3, a1, id(a3),id(a1), id(a3.arr), id(a1.arr)) # [6,2,3], [6,2,3], two different ids, two different ids
a3.arr[0] = 5
print(a3,a1,id(a3),id(a1),id(a3.arr),id(a1.arr)) # [5,2,3], [6,2,3], two different ids, two different ids
a1.arr[0] = 7
print(a3,a1,id(a3),id(a1),id(a3.arr),id(a1.arr)) # [5,2,3], [7,2,3], two differend ids, two different ids

[1 2 3]
2411152078096 2411156012944
[4 2 3] 2411152078096 2411156012944
[4 2 3] 2411157089232 2411156005456 2411152078096 2411156012944
[8 2 3] 2411157089232 2411156005456 [4 2 3] 2411152078096 2411156012944
[8 2 3] 2411157089232 2411156005456 [6 2 3] 2411152078096 2411156012944
[6 2 3] [6 2 3] 2411157019344 2411152078096 2411153055664 2411156012944
[5 2 3] [6 2 3] 2411157019344 2411152078096 2411153055664 2411156012944
[5 2 3] [7 2 3] 2411157019344 2411152078096 2411153055664 2411156012944
